# Importing Libraries

In [ ]:
import os
import re
import glob
import pathlib
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import PIL
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from collections import Counter

from warnings import filterwarnings
filterwarnings('ignore')

SEED=123
np.random.seed(SEED)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, Activation, 
    MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D, GlobalMaxPooling2D, add
)

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Creating a VGG19 model

In [ ]:
vgg_19 = VGG19(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

# Freeze all the layers, as non-trainable
for layer in vgg_19.layers[:]:
    layer.trainable = False

#Model summary
vgg_19.summary()

In [ ]:
model = Sequential()

# Add the vgg base 
model.add(vgg_19)
print("Shape:",model.output_shape)

# Add new layers
model.add(Flatten())
print("Shape:",model.output_shape)

model.add(Dense(1, activation='sigmoid'))
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# Compiling the model

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
datagen= image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Setting the directories for training

In [ ]:
train_generator=datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='training'
)

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator = datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='validation')

In [ ]:
validation_generator.class_indices

In [ ]:
nb_train_samples = 22018
nb_validation_samples = 5510
batch_size=16

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // (batch_size*6),
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

### The accuracy didnt change much so we are gonna try with different pre-trained models

# Creating a RESNET model

In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

# Freeze all the layers, as non-trainable
for layer in resnet.layers[:]:
    layer.trainable = False

#Model summary
resnet.summary()

In [ ]:
model = Sequential()

# Add the resnet 
model.add(resnet)
print("Shape:",model.output_shape)

# Add new layers
model.add(Flatten())
print("Shape:",model.output_shape)

model.add(Dense(1, activation='sigmoid'))
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# Setting the directories for training and compiling the model

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])
datagen= image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator=datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='training'
)
validation_generator = datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='validation')
nb_train_samples = 22018
nb_validation_samples = 5510
batch_size=16

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // (batch_size),
    epochs = 15,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
scores = model.evaluate_generator(validation_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.save('model_mr.h5')

# Creating an inceptionV3 model

In [ ]:
iv = InceptionV3(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

# Freeze all the layers, as non-trainable
for layer in iv.layers[:]:
    layer.trainable = False

#Model summary
iv.summary()

In [ ]:
model = Sequential()

# Add the iv
model.add(iv)
print("Shape:",model.output_shape)

# Add new layers
model.add(Flatten())
print("Shape:",model.output_shape)

model.add(Dense(1, activation='sigmoid'))
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# Setting the directories for training and compiling the model

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])
datagen= image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator=datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='training'
)
validation_generator = datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='validation')
nb_train_samples = 22018
nb_validation_samples = 5510
batch_size=16

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // (batch_size),
    epochs = 15,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
scores = model.evaluate_generator(validation_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.save('model_m_iv.h5')

# Creating our own CNN model

In [ ]:
model = Sequential()

#adding convo-pool layers
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(224,224,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(224,224,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=(224,224,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3),input_shape=(224,224,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

# flattening image
model.add(Flatten())

# adding dense layers
model.add(Dense(128,activation='relu'))
# adding dropout to minimize overfitting issue
model.add(Dropout(0.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

#compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=["accuracy"])


In [ ]:
model.summary()

# Setting the directories for training and compiling the model

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])
datagen= image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator=datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='training'
)
validation_generator = datagen.flow_from_directory(
    directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(224,224),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='validation')
nb_train_samples = 22018
nb_validation_samples = 5510
batch_size=16

# Setting the early stopping and running the model

In [ ]:
early_stop = EarlyStopping(monitor="val_loss",patience=5, verbose=True)

In [ ]:
history = model.fit(train_generator, steps_per_epoch=689 ,validation_data=validation_generator,epochs=20,callbacks=[early_stop],verbose=True)

In [ ]:
scores = model.evaluate_generator(validation_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.save('model_m.h5')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model,show_shapes = True).create(prog='dot', format='svg'))

# Plotting the accuracy and loss curves

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.legend(['validation'], loc='upper left')
plt.show()
